In [1]:
import pandas as pd
import numpy as np

In [2]:
from sklearn.model_selection import train_test_split
import tensorflow as tf
tfk = tf.keras
tfkl = tf.keras.layers
seed = 127
from matplotlib import pyplot as plt

Load Data

In [3]:
df = pd.read_csv("../data/train.csv")
df.head()

,text,label
0,The court granted by a 5-4 vote a request made...,real
1,""" Pennsylvania was a crucial swing state in th...",real
2,The company today is rolling out an update to ...,fake
3,"When it comes to trade policy, Hillary Clinton...",real
4,S. stocks had their worst April start since 19...,real


In [143]:
true_news = pd.read_csv("../data/True.csv")
fake_news = pd.read_csv("../data/Fake.csv")
true_news.head()

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


In [89]:
fake_news.head()

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [90]:
true_news["text"][0]

'WASHINGTON (Reuters) - The head of a conservative Republican faction in the U.S. Congress, who voted this month for a huge expansion of the national debt to pay for tax cuts, called himself a “fiscal conservative” on Sunday and urged budget restraint in 2018. In keeping with a sharp pivot under way among Republicans, U.S. Representative Mark Meadows, speaking on CBS’ “Face the Nation,” drew a hard line on federal spending, which lawmakers are bracing to do battle over in January. When they return from the holidays on Wednesday, lawmakers will begin trying to pass a federal budget in a fight likely to be linked to other issues, such as immigration policy, even as the November congressional election campaigns approach in which Republicans will seek to keep control of Congress. President Donald Trump and his Republicans want a big budget increase in military spending, while Democrats also want proportional increases for non-defense “discretionary” spending on programs that support educat

In [91]:
true_news["title"][3]

'FBI Russia probe helped by Australian diplomat tip-off: NYT'

In [92]:
true_news.shape, df.shape, fake_news.shape

((21417, 4), (232003, 2), (23481, 4))

In [15]:
bbc_news = pd.read_csv("../../data/real_bbc.csv", encoding = "ISO-8859-1")

In [17]:
bbc_news.head()

,news,type
0,China had role in Yukos split-up\n \n China le...,business
1,Oil rebounds from weather effect\n \n Oil pric...,business
2,Indonesia 'declines debt freeze'\n \n Indonesi...,business
3,$1m payoff for former Shell boss\n \n Shell is...,business
4,US bank in $515m SEC settlement\n \n Five Bank...,business


In [50]:
bbc_news.shape

(2225, 2)

In [19]:
bbc_news["news"][4]
#already received some kind of preprocessing

'US bank in $515m SEC settlement\n \n Five Bank of America subsidiaries have agreed to pay a total of $515m (Â£277m) to settle an investigation into fraudulent trading share practices.\n \n The US Securities and Exchange Commission announced the settlements, the latest in an industry-wide clean-up of US mutual funds. The SEC also said it had brought fraud charges against two ex-senior executives of Columbia Distributor. Columbia Distributor was part of FleetBoston, bought by BOA last year. Three other ex-Columbia executives agreed settlements with the SEC.\n \n The SEC has set itself the task of stamping out the mutual funds\' use of market-timing, a form of quick-fire, short-term share trading that harms the interests of small investors, with whom mutual funds are particularly popular. In the last two years, it has imposed penalties totalling nearly $2bn on 15 funds. The SEC unveiled two separate settlements, one covering BOA\'s direct subsidiaries, and another for businesses that wer

Create Dataset

In [144]:
true_news.drop(columns=["title","subject","date"], inplace = True)
fake_news.drop(columns=["title","subject","date"], inplace = True)
fake_news.head()

,text
0,Donald Trump just couldn t wish all Americans ...
1,House Intelligence Committee Chairman Devin Nu...
2,"On Friday, it was revealed that former Milwauk..."
3,"On Christmas day, Donald Trump announced that ..."
4,Pope Francis used his annual Christmas Day mes...


In [145]:
true_news['label'] = 0
fake_news['label'] = 1
df['label'][df['label']=='real'] = 0
df['label'][df['label']=='fake'] = 1

true_news.head()

,text,label
0,WASHINGTON (Reuters) - The head of a conservat...,0
1,WASHINGTON (Reuters) - Transgender people will...,0
2,WASHINGTON (Reuters) - The special counsel inv...,0
3,WASHINGTON (Reuters) - Trump campaign adviser ...,0
4,SEATTLE/WASHINGTON (Reuters) - President Donal...,0


In [ ]:
#Remove Reuters
import re
for i in range(len(true_news['text'])):
    true_news['text'][i] = re.sub('^.*?- ', '', true_news['text'][i])

# remove anything until -
# re.sub('^.*?- ', '', in_str)

In [147]:
true_news.head()

,text,label
0,The head of a conservative Republican faction ...,0
1,Transgender people will be allowed for the fir...,0
2,The special counsel investigation of links bet...,0
3,Trump campaign adviser George Papadopoulos tol...,0
4,President Donald Trump called on the U.S. Post...,0


In [148]:
df = df.append(true_news, ignore_index=True)
df = df.append(fake_news, ignore_index=True)

In [149]:
df.head()

,text,label
0,The court granted by a 5-4 vote a request made...,0
1,""" Pennsylvania was a crucial swing state in th...",0
2,The company today is rolling out an update to ...,1
3,"When it comes to trade policy, Hillary Clinton...",0
4,S. stocks had their worst April start since 19...,0


In [158]:
#shuffle
from sklearn.utils import shuffle
df = shuffle(df)
df.reset_index(inplace=True, drop=True)
df.head()

,text,label
0,The Tokyo Stock Exchange said on Monday shares...,1
1,"It steers clear of long-distance transport, wh...",1
2,Idi Inc * Idi announces 2016 second-quarter re...,1
3,The difference between us and them is that we...,1
4,Chuck Brown remembers the exact moment he was ...,1


In [159]:
df.to_csv(path_or_buf="../data/dataset.csv")

In [7]:
df = pd.read_csv("../../data/dataset.csv")

In [9]:
df.drop(columns="Unnamed: 0", inplace=True)

Train the Model

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

In [8]:
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [18]:
import re

In [19]:
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS = set(stopwords.words('english'))

def clean_text(text):
    """
        text: a string
        
        return: modified initial string
    """
    text = text.lower() # lowercase text
    text = REPLACE_BY_SPACE_RE.sub(' ', text) # replace REPLACE_BY_SPACE_RE symbols by space in text. substitute the matched string in REPLACE_BY_SPACE_RE with space.
    text = BAD_SYMBOLS_RE.sub('', text) # remove symbols which are in BAD_SYMBOLS_RE from text. substitute the matched string in BAD_SYMBOLS_RE with nothing. 
    text = ' '.join(word for word in text.split() if word not in STOPWORDS) # remove stopwors from text
    return text

df['text'] = df['text'].apply(clean_text)

In [20]:
df.head()

,text,label
0,tokyo stock exchange said monday shares takata...,1
1,steers clear longdistance transport traffics f...,1
2,idi inc idi announces 2016 secondquarter resul...,1
3,difference us listen says husband woman hellbe...,1
4,chuck brown remembers exact moment arrestedi s...,1


In [21]:
# The maximum number of words to be used. (most frequent)
MAX_NB_WORDS = 50000
# Max number of words in each news
MAX_SEQUENCE_LENGTH = 250
# This is fixed.
EMBEDDING_DIM = 100

tokenizer = Tokenizer(num_words=MAX_NB_WORDS,  # Max number of words to consider
                          filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',  # filter all punctuation except "'" (simplification)
                          lower=True,  # convert all words to lowercase
                          split=' ')  # Find words by splitting by space ' '

tokenizer.fit_on_texts(df["text"])
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 359887 unique tokens.


In [24]:
import pickle
# save the tokenizer to a file
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [177]:
# Compute number of words per sentence
words_per_sentence = [len(s.split(' ')) for s in df["text"]]
np.mean(words_per_sentence), np.max(words_per_sentence),  np.min(words_per_sentence)

(67.3952351201332, 4934, 1)

In [163]:
#X = ['SOS ' + s + ' EOS' for s in df['text']]

In [22]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [23]:
#Truncate and pad the input sequences so that they are all in the same length for modeling
X = tokenizer.texts_to_sequences(df['text'].values)
X = pad_sequences(X, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of data tensor:', X.shape)

Shape of data tensor: (276901, 250)


In [24]:
y = df['label']
y = y.astype(int)
y

0         1
1         1
2         1
3         1
4         1
         ..
276896    1
276897    0
276898    0
276899    0
276900    1
Name: label, Length: 276901, dtype: int32

In [25]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.10, random_state = 127)
print(X_train.shape,y_train.shape)
print(X_test.shape,y_test.shape)

(249210, 250) (249210,)
(27691, 250) (27691,)


In [26]:
X_train, X_val, y_train, y_val = train_test_split(X_train,y_train, test_size = 0.15, random_state = 127)
print(X_train.shape,y_train.shape)
print(X_val.shape,y_val.shape)

(211828, 250) (211828,)
(37382, 250) (37382,)


In [216]:
model = tfk.Sequential()
model.add(tfkl.Embedding(MAX_NB_WORDS, EMBEDDING_DIM, input_length=X.shape[1]))
model.add(tfkl.SpatialDropout1D(0.2))
model.add(tfkl.LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(tfkl.Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     (None, 250, 100)          5000000   
                                                                 
 spatial_dropout1d_4 (Spatia  (None, 250, 100)         0         
 lDropout1D)                                                     
                                                                 
 lstm_4 (LSTM)               (None, 100)               80400     
                                                                 
 dense_4 (Dense)             (None, 1)                 101       
                                                                 
Total params: 5,080,501
Trainable params: 5,080,501
Non-trainable params: 0
_________________________________________________________________


In [221]:
epochs = 5
batch_size = 128

In [ ]:
history = model.fit(x = X_train, 
                    y = y_train,
                     epochs=epochs, 
                     validation_data = (X_val,y_val),
                     batch_size= batch_size,
                     callbacks=[tfk.callbacks.EarlyStopping(monitor='val_accuracy', patience=20, restore_best_weights=True)])

In [26]:
accuracy = model.evaluate(X_test,y_test)
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accuracy[0],accuracy[1]))

866/866 [==============================] - 22s 24ms/step - loss: 0.1286 - accuracy: 0.9493
Test set
  Loss: 0.129
  Accuracy: 0.949


In [ ]:
plt.title('Loss')
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='test')
plt.legend()
plt.show()

In [ ]:
plt.title('Accuracy')
plt.plot(history.history['acc'], label='train')
plt.plot(history.history['val_acc'], label='test')
plt.legend()
plt.show()

In [ ]:
#model.save("LSTM_model")

In [16]:
model = tfk.models.load_model("LSTM_model")

In [7]:
# load the tokenizer from the file
import pickle
with open('tokenizer.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)

In [8]:
print(tf.__version__)

2.11.0


In [11]:
test = pd.read_csv("../../data/test.csv")

In [12]:
test["text"][0]

'We asked for "disclosure of any information that may be relevant to the investigation."\n\nA spokesman for the Department of Justice declined to comment on the probe, saying only that it was ongoing.'

In [13]:
test.head()

,text
0,"We asked for ""disclosure of any information th..."
1,Continued disruptions by a range of local grou...
2,Criminal gangs in China are faking outbreaks o...
3,"""After we announced the Hess transaction, we h..."
4,A Syngenta spokesman clarified his comment ear...


In [90]:
np.array(sum(seq, [])).shape

(132,)

In [102]:
test["text"][0:1]

0    We asked for "disclosure of any information th...
Name: text, dtype: object

In [118]:
new = 'We asked for "disclosure of any information that may be relevant to the investigation."\n\nA spokesman for the Department of Justice declined to comment on the probe, saying only that it was ongoing.'

In [ ]:
text = new.lower() 
text = REPLACE_BY_SPACE_RE.sub(' ', text) 
text = BAD_SYMBOLS_RE.sub('', text) 
text = ' '.join(word for word in text.split() if word not in STOPWORDS)
text = [text,'']
seq = tokenizer.texts_to_sequences(text)
padded = pad_sequences(seq, maxlen=250)
pred = model.predict(padded)[0][0]
print(pred)

In [27]:
bbc_news.head()

,news,type
0,China had role in Yukos split-up\n \n China le...,business
1,Oil rebounds from weather effect\n \n Oil pric...,business
2,Indonesia 'declines debt freeze'\n \n Indonesi...,business
3,$1m payoff for former Shell boss\n \n Shell is...,business
4,US bank in $515m SEC settlement\n \n Five Bank...,business


In [28]:
#Test on bbc news dataset (all news are true)
#Truncate and pad the input sequences so that they are all in the same length for modeling
X = tokenizer.texts_to_sequences(bbc_news['news'].values)
X = pad_sequences(X, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of data tensor:', X.shape)

Shape of data tensor: (2225, 250)


In [39]:
pred = model.evaluate(X,np.ones(X.shape[0]))


70/70 [==============================] - 2s 25ms/step - loss: 0.0998 - accuracy: 0.9627
